# Record Embedding

In [34]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import calendar
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from keras.models import load_model
import h5py
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn import metrics
from gensim.models import KeyedVectors


import gensim
from gensim.models import FastText

%matplotlib inline

## Read data
Details here: https://fasttext.cc/

In [35]:
df_truth = pd.read_csv('truth_values_1100_tuples.csv',dtype=object, encoding='utf8')
df_dirty = pd.read_csv('HospitalErrorsWithoutNan.csv',dtype=object, encoding='utf8')
df_dirty.drop(columns=['label'], axis=1, inplace=True)

In [36]:
df_truth.head(2)

,ProviderNumber,HospitalName,Address1,City,ZipCode,CountyName,PhoneNumber,HospitalOwner,Condition,Sample
0,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,35233,JEFFERSON,2053258100,Voluntary non-profit - Private,Surgical Infection Prevention,0 patients
1,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,35233,JEFFERSON,2053258100,Voluntary non-profit - Private,Surgical Infection Prevention,0 patients


In [37]:
df_dirty.head(2)

,ProviderNumber,HospitalName,Address1,City,ZipCode,CountyName,PhoneNumber,HospitalOwner,Condition,Sample
0,10011,ST VINCENP'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,35235,JEFFERSON,2058383122,Voluntary non-profit - Private,Pneumonia,69 patients
1,10027,ELBA GENERAL HOSPITAL,987 DRAYTON STREET,ELBA,36323,COFFEE,3348972257,Voluntary non-profit - Other,Surgical Infection Prevention,0 patients


## Preprocessing

In [38]:
combined_hosp = df_truth.values.tolist()
combined_dirty = df_dirty.values.tolist()

In [39]:
columns = df_truth.columns.values

In [40]:
ProviderNumberSet = set(df_truth['ProviderNumber'].tolist())
HospitalNameSet = set(df_truth['HospitalName'].tolist())
AddressSet = set(df_truth['Address1'].tolist())
CitySet = set(df_truth['City'].tolist())
ZipCodeSet = set(df_truth['ZipCode'].tolist())
CountyNameSet = set(df_truth['CountyName'].tolist())
PhoneNumberSet = set(df_truth['PhoneNumber'].tolist())
HospitalOwnerSet = set(df_truth['HospitalOwner'].tolist())
ConditionSet = set(df_truth['Condition'].tolist())
SampleSet = set(df_truth['Sample'].tolist())

In [41]:
combinedSet = [ProviderNumberSet, HospitalNameSet, AddressSet, CitySet, ZipCodeSet, CountyNameSet, PhoneNumberSet, HospitalOwnerSet, ConditionSet, SampleSet]

In [42]:
columns

array(['ProviderNumber', 'HospitalName', 'Address1', 'City', 'ZipCode',
       'CountyName', 'PhoneNumber', 'HospitalOwner', 'Condition',
       'Sample'], dtype=object)

## Model

In [53]:
path = 'HospitalMultiAttributeFastText.w2v'
fastTextModel = KeyedVectors.load(path)

In [54]:
model = load_model('HospitalMultiAttributeClassifier.h5')
with open('HospitalMultiAttributeClassifierTokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [55]:
mapping = dict(enumerate(sorted(df_truth.columns.values)))

In [56]:
mapping

{0: 'Address1',
 1: 'City',
 2: 'Condition',
 3: 'CountyName',
 4: 'HospitalName',
 5: 'HospitalOwner',
 6: 'PhoneNumber',
 7: 'ProviderNumber',
 8: 'Sample',
 9: 'ZipCode'}

## Correction Pipeline

In [171]:
def predictAttribute(model, tokenizer, value):
    """
    Classifies value parameter as its corresponding attribute
    """
    value = [value]
    sequences = tokenizer.texts_to_sequences(value)
    testData = pad_sequences(sequences, maxlen=45)
    predictions = model.predict(testData)
    return mapping[np.argmax(predictions[0])]

In [172]:
def correctCell(fastTextModel, model, tokenizer, row, topN=10):
    cellValues = {}
    output = dict()
    isMistake = False
    for cellIndex in range(len(row)):
        currentCellValue = row[cellIndex]
        if not currentCellValue in combinedSet[cellIndex]:
            isMistake = True
            cellValues['mistakeDetected'] = currentCellValue
            predictions = fastTextModel.most_similar(currentCellValue, topn=topN)
            for match, confidence in predictions:
                # Predicted type to be equal to the missing value
                if predictAttribute(model, tokenizer, match) == columns[cellIndex]:
                    if match in output and confidence <= output[match]:
                        continue
                    else:
                        output[match] = confidence
    
    if isMistake:
        results = output.items()
        predictedValue = sorted(results, key=lambda x: x[1], reverse=True)[0] if results else None
        cellValues['predictedValue'] = predictedValue[0] if predictedValue else None
        return cellValues
    else:
        return None

## Verification pipeline

In [128]:
dfUnique = df_truth.drop_duplicates()

In [129]:
len(dfUnique)

824

In [130]:
queryMapping = {
    "ProviderNumber" : 'HospitalName=="{}" and Address1=="{}" and City=="{}" and ZipCode=="{}" and CountyName=="{}"and PhoneNumber=="{}" and HospitalOwner == "{}" and Condition == "{}" and Sample=="{}"',
    "HospitalName" : 'ProviderNumber=="{}" and Address1=="{}" and City=="{}" and ZipCode=="{}" and CountyName=="{}"and PhoneNumber=="{}" and HospitalOwner == "{}" and Condition == "{}" and Sample=="{}"',
    "Address1" : 'ProviderNumber=="{}" and HospitalName=="{}" and City=="{}" and ZipCode=="{}" and CountyName=="{}"and PhoneNumber=="{}" and HospitalOwner == "{}" and Condition == "{}" and Sample=="{}"',
    "City" : 'ProviderNumber=="{}" and HospitalName=="{}" and Address1=="{}" and ZipCode=="{}" and CountyName=="{}"and PhoneNumber=="{}" and HospitalOwner == "{}" and Condition == "{}" and Sample=="{}"',
    "ZipCode" : 'ProviderNumber=="{}" and HospitalName=="{}" and Address1=="{}" and City=="{}" and CountyName=="{}"and PhoneNumber=="{}" and HospitalOwner == "{}" and Condition == "{}" and Sample=="{}"',
    "CountyName" : 'ProviderNumber=="{}" and HospitalName=="{}" and Address1=="{}" and City=="{}" and ZipCode=="{}" and PhoneNumber=="{}" and HospitalOwner == "{}" and Condition == "{}" and Sample=="{}"',
    "PhoneNumber" : 'ProviderNumber=="{}" and HospitalName=="{}" and Address1=="{}" and City=="{}" and ZipCode=="{}" and CountyName=="{}" and HospitalOwner == "{}" and Condition == "{}" and Sample=="{}"',
    "HospitalOwner" : 'ProviderNumber=="{}" and HospitalName=="{}" and Address1=="{}" and City=="{}" and ZipCode=="{}" and CountyName=="{}" and PhoneNumber=="{}" and Condition == "{}" and Sample=="{}"',
    "Condition" : 'ProviderNumber=="{}" and HospitalName=="{}" and Address1=="{}" and City=="{}" and ZipCode=="{}" and CountyName=="{}"and PhoneNumber=="{}" and HospitalOwner == "{}" and Sample=="{}"',
    "Sample" : 'ProviderNumber=="{}" and HospitalName=="{}" and Address1=="{}" and City=="{}" and ZipCode=="{}" and CountyName=="{}"and PhoneNumber=="{}" and HospitalOwner == "{}" and Condition == "{}"',
}

In [169]:
correct = 0
inCorrect = 0
incorrectPredictions = []

In [170]:
for row in combined_dirty:
    output = correctCell(fastTextModel, model, tokenizer, row, 15)
    if output:
        detectedError = output['mistakeDetected']
        predictedValue = output['predictedValue']
    
        tempRow = row[:]
        errorIndex = row.index(detectedError)
        tempRow.remove(detectedError)

        # Run the query
        query = (queryMapping[columns[errorIndex]]).format(*tempRow)
        outputDf = dfUnique.query(query)
        actual = outputDf.head(1).values.tolist()[0][errorIndex]

        if actual == predictedValue:
            correct += 1
        else:
            inCorrect += 1
            incorrectPredictions.append((actual, predictedValue))

## Results

In [173]:
correct

186

In [160]:
inCorrect

7

In [161]:
incorrectPredictions

[('MONTGOMERY', 'BUTLER'),
 ('FAYETTE', 'ETOWAH'),
 ('MONTGOMERY', 'BUTLER'),
 ('MONTGOMERY', 'BUTLER'),
 ('FAYETTE', 'MADISON'),
 ('MOBILE', 'DALE'),
 ('CULLMAN', 'MADISON')]

In [162]:
accuracy = correct / (correct + inCorrect) * 1.0

In [163]:
accuracy

0.9637305699481865